# 광학 인식

![신문을 읽고 있는 로봇](./images/ocr.jpg)

흔히 볼 수 있는 Computer Vision 과제는 이미지에서 텍스트를 감지하고 해석하는 것입니다. 이러한 종류의 처리를 종종 *OCR(광학 인식)*이라고 합니다.

## Computer Vision 서비스를 사용하여 이미지에서 텍스트 읽기

**Computer Vision** Cognitive Service는 다음을 비롯한 OCR 작업을 지원합니다.

- 여러 언어로 된 텍스트를 읽는 데 사용할 수 있는 **OCR** API. 이 API는 동기식으로 사용할 수 있으며, 이미지에서 소량의 텍스트를 감지하고 읽어야 할 때 잘 작동합니다.
- 더 큰 문서에 최적화된 **Read** API. 이 API는 비동기식으로 사용되며, 인쇄 텍스트와 필기 텍스트 모두에 사용할 수 있습니다.

이 서비스는 **Computer Vision** 리소스 또는 **Cognitive Services** 리소스를 만들어서 사용할 수 있습니다.

아직 만들지 않았다면 Azure 구독에서 **Cognitive Services** 리소스를 만듭니다.

> **참고**: 이미 Cognitive Services 리소스를 보유하고 있다면 Azure Portal에서 **빠른 시작** 페이지를 열고 키 및 엔드포인트를 아래의 셀로 복사하기만 하면 됩니다. 리소스가 없다면 아래의 단계를 따라 리소스를 만듭니다.

1. 다른 브라우저 탭에서 Azure Portal(https://portal.azure.com) 을 열고 Microsoft 계정으로 로그인합니다.

2. **&#65291;리소스 만들기** 단추를 클릭하고, *Cognitive Services*를 검색하고, 다음 설정을 사용하여 **Cognitive Services** 리소스를 만듭니다.
    - **구독**: *사용자의 Azure 구독*.
    - **리소스 그룹**: *고유한 이름의 새 리소스 그룹을 선택하거나 만듭니다*.
    - **지역**: *사용 가능한 지역을 선택합니다*.
    - **이름**: *고유한 이름을 입력합니다*.
    - **가격 책정 계층**: S0
    - **알림을 읽고 이해했음을 확인합니다**. 선택됨.
3. 배포가 완료될 때까지 기다립니다. 그런 다음에 Cognitive Services 리소스로 이동하고, **개요** 페이지에서 링크를 클릭하여 서비스의 키를 관리합니다. 클라이언트 애플리케이션에서 Cognitive Services 리소스에 연결하려면 엔드포인트 및 키가 필요합니다.

### Cognitive Services 리소스의 키 및 엔드포인트 가져오기

Cognitive Services 리소스를 사용하려면 클라이언트 애플리케이션에 해당 엔드포인트 및 인증 키가 필요합니다.

1. Azure Portal에 있는 Cognitive Service 리소스의 **키 및 엔드포인트** 페이지에서 리소스의 **Key1**을 복사하고 아래 코드에 붙여 넣어 **YOUR_COG_KEY**를 대체합니다.
2. 리소스의 **엔드포인트**를 복사하고 아래 코드에 붙여 넣어 **YOUR_COG_ENDPOINT**를 대체합니다.
3. **셀 실행**(&#9655;) 단추(셀 왼쪽에 있음)를 클릭하여 아래의 셀에 있는 코드를 실행합니다.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

이제 키와 엔드포인트를 설정했으므로 Computer Vision 서비스 리소스를 사용하여 이미지에서 텍스트를 추출할 수 있습니다.

먼저, 이미지를 동기식으로 분석하고 포함된 텍스트를 읽을 수 있게 해주는 **OCR** API부터 시작하겠습니다. 이 경우에는 일부 텍스트를 포함하고 있는 가상의 Northwind Traders 소매업체에 대한 광고 이미지가 있습니다. 아래의 셀을 실행하여 읽어 보세요. 

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

이미지에 있는 텍스트는 영역, 줄, 단어의 계층 구조로 구성되어 있으며 코드는 이 항목들을 읽어서 결과를 검색합니다.

이미지 위에서 읽은 텍스트를 결과에서 봅니다. 

## 경계 상자 표시

텍스트 줄의 *경계 상자* 좌표와 이미지에서 발견된 개별 단어도 결과에 포함되어 있습니다. 아래의 셀을 실행하여 위에서 검색한 광고 이미지에서 텍스트 줄의 경계 상자를 확인하세요.

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

결과에서 각 텍스트 줄의 경계 상자는 이미지에 직사각형으로 표시됩니다.

## Read API 사용

이전에 사용한 OCR API는 소량의 텍스트가 있는 이미지에서 잘 작동합니다. 스캔한 문서와 같이 더 큰 텍스트 본문을 읽어야 할 때는 **Read** API를 사용할 수 있습니다. 이를 위해서는 다단계 프로세스가 필요합니다.

1. 비동기식으로 읽고 분석할 이미지를 Computer Vision 서비스에 제출합니다.
2. 분석 작업이 완료될 때까지 기다립니다.
3. 분석의 결과를 검색합니다.

이 프로세스를 사용하여 스캔한 서신의 텍스트를 Northwind Traders 매장 관리자에게 읽어 주려면 다음 셀을 실행하세요.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

결과를 검토합니다. 서신의 전체 필사본이 있는데, 대부분은 인쇄된 텍스트이고 필기 서명이 있습니다. 서신의 원본 이미지는 OCR 결과 아래에 표시됩니다(보기 위해 스크롤해야 할 수도 있음).

## 필기 텍스트 읽기

이전 예에서 이미지 분석 요청은 *인쇄된* 텍스트에 맞춰 작업을 최적화하는 텍스트 인식 모드를 지정했습니다. 그럼에도 불구하고 필기 서명이 읽혔습니다.

필기 텍스트를 읽을 수 있는 이 능력은 매우 유용합니다. 예를 들어 쇼핑 목록이 포함된 메모를 작성했는데 폰의 앱을 사용하여 메모를 읽고 그 안에 포함된 텍스트를 필사하기를 원한다고 가정해 보세요.

아래 셀을 실행하여 필기 쇼핑 목록에 대한 읽기 작업의 예를 확인해 보세요.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## 추가 정보

OCR에 Computer Vision 서비스를 사용하는 방법에 대한 자세한 내용은 [Computer Vision 설명서](https://docs.microsoft.com/ko-kr/azure/cognitive-services/computer-vision/concept-recognizing-text)를 참조하세요.